<a href="https://colab.research.google.com/github/sseyonafre/Project/blob/main/sprint2/AI_14_%E1%84%80%E1%85%A9%E1%84%89%E1%85%A6%E1%84%92%E1%85%B4_Section2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *DATA SCIENCE / SECTION 2 / PROJECT*

# 🏆 프로젝트 목표
>타격데이터를 통한 2022년 프로야구 승패예측모델 만들기

#  ⚾️ Data Description


#  데이터 탐색


## **1. Gathering Data**

In [ ]:
#https://www.kaggle.com/datasets/park123/korea-baseball-datasetkbo-20152020
#여기서 2015~2020년까지의 데이터 수집한 후, 2021,22년의 데이터는 여기 캐글에 올려진 깃헙에서 연도만 바꿔서 추가수집. 

In [1]:
import pandas as pd
import numpy as np
from google.colab import files
uploaded = files.upload()

Saving kbo.csv to kbo.csv


## **2. 데이터 탐색 및 정제**

In [ ]:
#n233a 에 정제함 
#concat()이용하여 2015년부터 2022년 데이터를 전부 합쳐주고 , game_id도 새로만들고, 컬럼이름도 수정하고, 팀이름도 수정하고 필요없는 컬럼 드랍까지 완료.

#Feature Engineering

In [ ]:
#n233a 에 feature engineering 했음.